In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params, dist_kernel
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import unet_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'unet_dist'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 30
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train_original, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train_original, y_val_original = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

In [4]:
y_base_train = np.array([dist_kernel(y) for y in y_base_train_original])
y_train = np.array([dist_kernel(y) for y in y_train_original])
y_val = np.array([dist_kernel(y) for y in y_val_original])

y_base_train.shape, y_train.shape, y_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [5]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_unet_depth = hp.Int("unet_depth", min_value=0, max_value=3, step=1, default=2)
    hp_n_conv_layers = hp.Int("n_conv_layers", min_value=1, max_value=3, step=1)
    hp_filters_mult = hp.Choice("conv_filters_mult", values=[1, 2, 4, 8, 16], default=4)
    hp_spatial_dropout = hp.Choice("conv_spatial_dropout", values=[0.0, 0.1, 0.2])
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_normalize_signal = hp.Boolean("normalize_signal", default=False)

    model = bare_model_builder(hp_unet_depth, hp_n_conv_layers, hp_filters_mult, hp_spatial_dropout,
                               hp_batch_normalization, hp_input_batch_normalization, hp_normalize_signal)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [6]:
model_builder(kt.HyperParameters()).summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24)]         0           []                               
                                                                                                  
 reshape (Reshape)              (None, 24, 1)        0           ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 24, 32)       96          ['reshape[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 12, 32)       0           ['conv1d[0][0]']                 
                                                                                              

In [7]:
model_callbacks = [
    callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)
]

# Bayesian tuner

In [8]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [9]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False}
Number of parameters: 367041
========== Model 1 ==========
{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 1472481
========== Model 2 ==========
{'unet_depth': 3, 'n_conv_layers': 3, 'conv_filters_mult': 16, 'conv_spatial_dropout': 0.0, 'batch_normalization': True, 'input_batch_normalization': False, 'normalize_signal': False}
Number of parameters: 7598593
========== Model 3 ==========
{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': True}
Number of parameters: 1472481
========== Model 4 ==========


# Cross-validation for top 5 models

In [10]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME,
                                           n_epochs=N_EPOCHS, batch_size=BATCH_SIZE, n_top=TOP_N,
                                           es_min_delta=ES_MIN_DELTA, n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC,
                                           overwrite=OVERWRITE)
model_scores = cross_validator()

{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 4, 'conv_spatial_dropout': 0.0, 'batch_normalization': False, 'input_batch_normalization': False, 'normalize_signal': False}
Number of parameters: 367041
Got score: 9.0285 (8.8978, 9.1591)
Got score: 8.8895 (8.7549, 9.0241)
Got score: 8.5044 (8.5242, 8.4845)
Got score: 8.9919 (8.9611, 9.0226)
Got score: 9.0527 (9.0561, 9.0493)


{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': False}
Number of parameters: 1472481
Got score: 9.8285 (10.1374, 9.5196)
Got score: 9.7909 (9.1827, 10.3991)
Got score: 8.9730 (8.9352, 9.0109)
Got score: 8.8966 (8.7998, 8.9934)
Got score: 10.7110 (9.8018, 11.6202)


{'unet_depth': 3, 'n_conv_layers': 3, 'conv_filters_mult': 16, 'conv_spatial_dropout': 0.0, 'batch_normalization': True, 'input_batch_normalization': False, 'normalize_signal': False}
Number of parameters: 7598593
Got score: 9.2927 (9.1774, 9.4080)
Got score: 10.1577 (8.7090, 11.6064)
Got score: 170.3029 (332.0369, 8.5689)
Got score: 90.7711 (172.8392, 8.7030)
Got score: 9.9830 (10.2698, 9.6962)


{'unet_depth': 3, 'n_conv_layers': 2, 'conv_filters_mult': 8, 'conv_spatial_dropout': 0.1, 'batch_normalization': True, 'input_batch_normalization': True, 'normalize_signal': True}
Number of parameters: 1472481
Got score: 9.8134 (10.0428, 9.5841)
Got score: 10.4775 (11.2808, 9.6741)
Got score: 9.2376 (9.3017, 9.1736)
Got score: 10.0263 (10.3171, 9.7355)
Got score: 10.1082 (10.5799, 9.6365)


{'unet_depth': 2, 'n_conv_layers': 3, 'conv_filters_mult': 16, 'conv_spatial_dropout': 0.0, 'batch_normalization': True, 'input_batch_normalization': False, 'normalize_signal': True}
Number of parameters: 1815041
Got score: 16.1759 (9.6624, 22.6893)
Got score: 23.1996 (11.2756, 35.1236)
Got score: 12.6634 (10.4133, 14.9136)
Got score: 31.2533 (53.1301, 9.3766)
Got score: 10.7976 (10.9326, 10.6625)


In [11]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.89,0.20,367041
1,9.64,0.66,1472481
2,58.10,64.27,7598593
3,9.93,0.41,1472481
4,18.82,7.52,1815041
